<a href="https://colab.research.google.com/github/akashwaf/finvasia01/blob/main/simple_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# run to install library temporarily 

!git clone https://github.com/Shoonya-Dev/ShoonyaApi-py.git

# this are the libraries mentioned in requirements.txt
!pip install requests
!pip install websocket_client
!pip install /content/ShoonyaApi-py/dist/NorenRestApiPy-0.0.20-py2.py3-none-any.whl
!pip install pyyaml


!pip install pyotp


# !pip install talib-binary             # recently stopped working 
# !pip install TA-Lib-binary               # working now 

Cloning into 'ShoonyaApi-py'...
remote: Enumerating objects: 287, done.
remote: Counting objects: 100% (287/287), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 287 (delta 153), reused 283 (delta 151), pack-reused 0
Receiving objects: 100% (287/287), 275.64 KiB | 7.88 MiB/s, done.
Resolving deltas: 100% (153/153), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./ShoonyaApi-py/dist/NorenRestApiPy-0.0.20-py2.py3-none-any.whl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

import multiprocessing as mp 
import time
import math
import os , sys
import logging
import pandas as pd 
import pprint
import time
import yaml
import warnings 

# manually installed imports 
import pyotp

In [3]:
import os, time
os.environ['TZ'] = "Asia/Calcutta" # set new timezone
time.tzset()

sold_ce = 1
sold_pe = 2

sold_ltp =  {
    
    sold_ce: 0000,
    sold_pe: 0000
  
}


premium = 25
sl = premium* 2
max_sl = premium* 4

expiry = '08-DEC-2022'

# making global variable and using it in other functions
token_df = pd.read_csv('https://shoonya.finvasia.com/NFO_symbols.txt.zip')

token_df  = token_df [token_df['Symbol'] == 'BANKNIFTY']  
token_df  = token_df [token_df['Expiry'] == expiry]
token_df.sort_values(by =  'StrikePrice', inplace = True)
global_token_df = token_df 


ce_token_df  = token_df [token_df['OptionType'] == 'CE']
ce_token_list  = ce_token_df[ 'Token'].tolist()
ce_token_str_list = [str(x) for x in ce_token_list]


pe_token_df  = token_df [token_df['OptionType'] == 'PE']
pe_token_list  = pe_token_df['Token'].tolist()
pe_token_str_list = [str(x) for x in pe_token_list]



ce_lp_data = {}
pe_lp_data = {}
lp_data ={}

feed_opened = False




In [4]:

if True: 
  # warnings.simplefilter("ignore")
  feed_opened = False
  feedJson = {}
  socket_opened = False
  orderJson = {}


  acknowledgement_data = {}
  lp_data = {}
  ce_lp_data = {}
  pe_lp_data = {}



  def event_handler_feed_update(tick_data):
      
      global data_response_type
      global ce_lp_data
      global pe_lp_data
      global lp_data 
      global sold_ltp





      # UPDATE LIVE LTP IN  GLOBAL JASON FORMAT VARIABLE 
      if 'lp' in tick_data:
        lp_data[tick_data['tk']] =   tick_data['lp']


        if tick_data['tk'] in ce_token_str_list :
          ce_lp_data [tick_data['tk']] = tick_data['lp']
        
        
        if tick_data['tk'] in pe_token_str_list :
          pe_lp_data [tick_data['tk']] = tick_data['lp']

             
        
        if tick_data['tk'] == sold_ce or tick_data['tk'] == sold_ce : 

          if tick_data['tk'] == sold_ce: 
            sold_ltp[sold_ce]  =  tick_data['lp']

            
          if tick_data['tk'] == sold_pe: 
            sold_ltp[sold_pe]  =  tick_data['lp']



  def event_handler_order_update(order_data):
      print('orderupdate')
      
      print(order_data)

  def open_callback():
      global feed_opened
      feed_opened = True


  def close_callback():
      global feed_opened
      feed_opened = False

  def setupWebSocket() :

    api.close_websocket()
    global fee_opened
    
    api.start_websocket( order_update_callback=event_handler_order_update,
                      subscribe_callback=event_handler_feed_update, 
                      socket_open_callback= open_callback,
                        socket_close_callback = close_callback)

    time.sleep(1)
    
    while(feed_opened==False):
        # print('here only')
        pass


    return True 



In [5]:
if True : 
  if True:
    from NorenRestApiPy.NorenApi import  NorenApi
    from threading import Timer
    import pandas as pd
    import time
    import concurrent.futures

    api = None
    class Order:
        def __init__(self, buy_or_sell:str = None, product_type:str = None,
                    exchange: str = None, tradingsymbol:str =None, 
                    price_type: str = None, quantity: int = None, 
                    price: float = None,trigger_price:float = None, discloseqty: int = 0,
                    retention:str = 'DAY', remarks: str = "tag",
                    order_id:str = None):
            self.buy_or_sell=buy_or_sell
            self.product_type=product_type
            self.exchange=exchange
            self.tradingsymbol=tradingsymbol
            self.quantity=quantity
            self.discloseqty=discloseqty
            self.price_type=price_type
            self.price=price
            self.trigger_price=trigger_price
            self.retention=retention
            self.remarks=remarks
            self.order_id=None


        #print(ret)

        


    def get_time(time_string):
        data = time.strptime(time_string,'%d-%m-%Y %H:%M:%S')

        return time.mktime(data)


    class ShoonyaApiPy(NorenApi):
        def __init__(self):
            NorenApi.__init__(self, host='https://api.shoonya.com/NorenWClientTP/', websocket='wss://api.shoonya.com/NorenWSTP/', eodhost='https://shoonya.finvasia.com/chartApi/getdata/')
            global api
            api = self

        def place_basket(self, orders):

            resp_err = 0
            resp_ok  = 0
            result   = []
            with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:

                future_to_url = {executor.submit(self.place_order, order): order for order in  orders}
                for future in concurrent.futures.as_completed(future_to_url):
                    url = future_to_url[future]
                try:
                    result.append(future.result())
                except Exception as exc:
                    print(exc)
                    resp_err = resp_err + 1
                else:
                    resp_ok = resp_ok + 1

            return result
                    
        def placeOrder(self,order: Order):
            ret = NorenApi.place_order(self, buy_or_sell=order.buy_or_sell, product_type=order.product_type,
                                exchange=order.exchange, tradingsymbol=order.tradingsymbol, 
                                quantity=order.quantity, discloseqty=order.discloseqty, price_type=order.price_type, 
                                price=order.price, trigger_price=order.trigger_price,
                                retention=order.retention, remarks=order.remarks)
            #print(ret)

            return ret





  def do_login():
    #enable dbug to see request and responses
    logging.basicConfig(level=logging.DEBUG)

    #start of our program
    api = ShoonyaApiPy()

    #credentials
    user        = 'FA13620'
    pwd       =  '2012Ak@sh'
    token =     "52R5K462BA4H5TJ7SS746S5534364527"
    factor2     =  pyotp.TOTP(token).now()
    vc          = 'FA13620_U'
    app_key     = '9088c808ac3522b35705a4acd8a46d03'
    imei        = 'abc1234'




    ret = api.login(userid=user, password=pwd, twoFA=factor2, vendor_code=vc, api_secret=app_key, imei=imei)


    print(factor2)

    print(ret)



  def get_token_df (opt_type= 'CE', expiry = '08-DEC-2022'):

    # read
        
    token_df = pd.read_csv('https://shoonya.finvasia.com/NFO_symbols.txt.zip')
    


    banknifty_token  = token_df [token_df['Symbol'] == 'BANKNIFTY']  
    banknifty_token  = banknifty_token [banknifty_token['Expiry'] == expiry]
    banknifty_token  = banknifty_token [banknifty_token['OptionType'] == opt_type]
    print(banknifty_token)

    banknifty_token.sort_values(by =  'StrikePrice', inplace = True)


    return banknifty_token




  def make_ready_to_subscribe( token_df  ):

    print(token_df)

    token_list = token_df['Token'].tolist()

    
    # token_list = token_df[80:len(token_df)]['Token'].tolist()

    ready_to_subscribe = []
    for i in token_list:

      ready_to_subscribe.append( 'NFO|'+str(i) )


    print(ready_to_subscribe)


    return ready_to_subscribe







  def select_opt_by_premium (premium  = 100):
    # returns token and price of the selected premium 
    # for that it looks into ce_lp_data which is updated by the websocket 

    diff  = 1000000
    nearest_price = 1000000
    nearest_token = 9000000000

    for x, y  in ce_lp_data.items():


      if abs(float(y) - premium) < diff :

        diff = abs(float(y) - premium)
        nearest_token = x
        nearest_price = float(y)


    
      # print(x , y )

    print('nearest token , nearest_price, and difference is ')
    print(nearest_token,nearest_price , diff)


    return nearest_token , nearest_price





  def set_simple_momentum(token , ltp):

    # set dictionary which will be checked continuously.
    global simple_momentum_check

    simple_momentum_check [token]  = {}

    simple_momentum_check[token]['level'] =  ltp - ltp *0.001 
    simple_momentum_check[token]['side'] =  'down'
    simple_momentum_check[token]['condition'] =  'False'
    simple_momentum_check[token]['action'] =  'sell'           
    simple_momentum_check[token]['action_successfull'] =  'NS'             # not successfull = NS , successfull = S
    simple_momentum_check[token]['close'] =  'N'                           # NO = N , YES = Y 

    return True 









  def continuous_checking ():
    # continuously checks for  condtion satisfied or not  i.e. simple momentum condition 



    global simple_momentum_check 


    counter = 0
    token_key_list  = list(simple_momentum_check.keys())


    print(len(simple_momentum_check))
    print(len(token_key_list))


    while len(simple_momentum_check) != 0:

      time.sleep(1)
      print(counter)

      token = token_key_list[counter]


      if  (float(ce_lp_data[token] )) <  simple_momentum_check[token] ['level'] :
        
          simple_momentum_check[token] ['action']  = True 

          print('take_some_acctions ')
          ######### take some actions ###### takes some trades ######### and get info updated 




      if counter +1 >= len(simple_momentum_check):
        counter = 0
      else:
        counter = counter + 1 

      
      time.sleep(1)
      





  # if True: 
  #   # warnings.simplefilter("ignore")
  #   feed_opened = False
  #   feedJson = {}
  #   socket_opened = False
  #   orderJson = {}


    

  #   def event_handler_feed_update(tick_data):
        
  #       global sold_ltp 

  #       if sold_ce in tick_data:          

  #         sold_ltp[ sold_ce] = tick_data['lp']

  #       if sold_pe in tick_data: 

  #         sold_ltp[ sold_pe] = tick_data [lp]




  #   def event_handler_order_update(order_data):
  #       print('orderupdate')
  #       print(order_data)

  #   def open_callback():
  #       global feed_opened
  #       feed_opened = True


  #   def close_callback():
  #       global feed_opened
  #       # feed_opened = False

  #   def setupWebSocket() :

  #     api.close_websocket()
  #     global fee_opened
      
  #     api.start_websocket( order_update_callback=event_handler_order_update,
  #                       subscribe_callback=event_handler_feed_update, 
  #                       socket_open_callback= open_callback,
  #                         socket_close_callback = close_callback)

  #     time.sleep(1)
      
  #     while(feed_opened==False):
  #         # print('here only')
  #         pass


  #     return True 




In [6]:

def select_opt_by_premium_type (premium , opt_type  ):
  # returns token and price of the selected premium 
  # for that it looks into ce_lp_data which is updated by the websocket 

  diff  = 1000000
  nearest_price = 1000000
  nearest_token = 9000000000

  global ce_lp_data
  global pe_lp_data
  
  if opt_type == 'CE':

    for x, y  in ce_lp_data.items():

      if abs(float(y) - premium) < diff :

        diff = abs(float(y) - premium)
        nearest_token = x
        nearest_price = float(y)
        # print(x , y )

    print('nearest token , nearest_price, and difference is ')
    print(nearest_token,nearest_price , diff)

    return nearest_token , nearest_price




  if opt_type == 'PE' :
        
    for x, y  in pe_lp_data.items():

        if abs(float(y) - premium) < diff :

          diff = abs(float(y) - premium)
          nearest_token = x
          nearest_price = float(y)

          # print(x , y )

    print('nearest token , nearest_price, and difference is ')
    print(nearest_token,nearest_price , diff)


    return nearest_token , nearest_price

  
  return 0 , 0


In [7]:
def token_to_tradingsymbol ( token ):



  print(' token_to_tradingsymbol:  token passed here is ' , token  )

  global global_token_df

  try : 
    trading_symbol = global_token_df [ global_token_df [ 'Token']  == token ]

    trading_symbol = trading_symbol ['TradingSymbol'].iloc[0]
    print(trading_symbol)

    return trading_symbol

  except IndexError as e :

    print('invalid token  ')
    return 0
  
  


token_to_tradingsymbol( int( '43308') )


 token_to_tradingsymbol:  token passed here is  43308
BANKNIFTY08DEC22C31500


'BANKNIFTY08DEC22C31500'

In [8]:
def exit_position( exit_token  ):

  exit_tradingsymbol = token_to_tradingsymbol ( int(exit_token)  )

  

  ret = api.place_order(buy_or_sell='B',
                        product_type='I',
                        exchange='NFO',
                        tradingsymbol= exit_tradingsymbol,
                        quantity=25,
                        discloseqty=0, 
                        price_type='MKT', 
                        # price=200.00, 
                        # trigger_price=199.50,
                        retention='DAY',
                        remarks='mkt_order')
  




  # exit position with exit_tradingsymbol
  pass




def make_position( new_premium ,  new_type ):

  new_token, new_price  = select_opt_by_premium_type ( premium =  new_premium ,opt_type  =  new_type)



  print(' in make position ,   new token and new_price', new_token, new_price)

  new_tradingsymbol = token_to_tradingsymbol ( int(new_token) )
  print('111111111111111')
  print(new_tradingsymbol)
  print(14512145512451245845)

  time.sleep(5)



  ret = api.place_order(buy_or_sell='S',
                        product_type='I',
                        exchange='NFO',
                        tradingsymbol= new_tradingsymbol,
                        quantity=25,
                        discloseqty=0, 
                        price_type='MKT', 
                        # price=200.00, 
                        # trigger_price=199.50,
                        retention='DAY',
                        remarks='mkt_order')
  
  print(ret)


  # place sell order with new_tradingsymbol ####################################

  
  if True :     # if order place successfully

    
    global sold_ltp 
    global sold_ce
    global sold_pe

    if new_type == 'CE': 
      del sold_ltp [ sold_ce]
      sold_ltp [new_token]  =   0
      sold_ce = new_token


    
    if new_type == 'PE': 
      del sold_ltp [ sold_pe]
      sold_ltp[ new_token] = 0
      sold_pe = new_token




In [9]:

do_login()

global token_df
ready_to_subscribe = make_ready_to_subscribe(token_df)


# it starts websocket 


global feed_opened 
if feed_opened == True :

  api.close_websocket()
  time.sleep(1)
  setupWebSocket()
else:
  setupWebSocket()
  feed_opened = True


time.sleep(5)




172721
{'request_time': '09:04:59 07-12-2022', 'actid': 'FA13620', 'uname': 'AKASH RAJESHKUMAR PATEL', 'prarr': [{'prd': 'C', 's_prdt_ali': 'CNC', 'exch': ['NSE', 'BSE', 'NIPO']}, {'prd': 'M', 's_prdt_ali': 'NRML', 'exch': ['NFO', 'CDS', 'BCD', 'MCX']}, {'prd': 'I', 's_prdt_ali': 'MIS', 'exch': ['NSE', 'BSE', 'NFO', 'CDS', 'BCD', 'MCX']}, {'prd': 'H', 's_prdt_ali': 'CO', 'exch': ['NSE', 'NFO', 'CDS', 'MCX', 'BSE']}, {'prd': 'B', 's_prdt_ali': 'BO', 'exch': ['NSE', 'NFO', 'CDS', 'MCX', 'BSE']}], 'stat': 'Ok', 'susertoken': '9f7a444be5fcfc98331ba77f2a4cfeae0b6a7d03aa9f1c4647bbf0f69442bfd0', 'email': 'AKSPMONEY1@GMAIL.COM', 'uid': 'FA13620', 'brnchid': 'HO', 'orarr': ['LMT', 'MKT', 'SL-LMT', 'SL-MKT'], 'exarr': ['NSE', 'NFO', 'CDS', 'MCX', 'BSE', 'BCD', 'NIPO'], 'values': ['1'], 'mws': {'1': []}, 'brkname': 'FINV', 'lastaccesstime': '1670384099'}
      Exchange  Token  LotSize     Symbol           TradingSymbol  \
82606      NFO  43308       25  BANKNIFTY  BANKNIFTY08DEC22C31500   
82605 

In [14]:


# api.unsubscribe(ready_to_subscribe)

api.subscribe(ready_to_subscribe, feed_type='t')



In [ ]:
# api.close_websocket()

In [25]:
if sold_ce == 1 and sold_pe  == 2  or True: 

   
  make_position ( new_premium =  premium  ,  new_type = 'CE' )
  make_position ( new_premium =  premium  ,  new_type = 'PE' )


  pass


while True:

  if float(sold_ltp [sold_ce])  > sl  or  float (sold_ltp[sold_pe]  ) > sl: 
    

    if sl == max_sl : 
      pass
      # exit all position and stop trading

      exit_position( sold_ce)
      exit_position (sold_pe) 

      sys.exit(0)




    if sold_ltp[sold_ce] > sl : 
      pass

      exit_position(  sold_pe   )

      make_position ( new_premium =  premium * 2  ,  new_type = 'PE' )

      # global premium 
      # premium = 2 * premium 
      # sl = premium * 2 

      
        # it will update sold_ce , sl , premium , sold_ltp 

    if sold_ltp[sold_pe] > sl : 
      pass
      
      exit_position(  sold_pe   )
      
      premium 
      make_position ( new_premium =  premium  * 2  ,  new_type = 'CE' )
      
      # global premium 
      # premium = 2 * premium 
      # sl = premium * 2 




nearest token , nearest_price, and difference is 
43778 26.0 1.0
 in make position ,   new token and new_price 43778 26.0
 token_to_tradingsymbol:  token passed here is  43778
BANKNIFTY08DEC22C43900
111111111111111
BANKNIFTY08DEC22C43900
14512145512451245845
{'request_time': '09:19:33 07-12-2022', 'stat': 'Ok', 'norenordno': '22120700030420'}
nearest token , nearest_price, and difference is 
43671 22.4 2.6000000000000014
 in make position ,   new token and new_price 43671 22.4
 token_to_tradingsymbol:  token passed here is  43671
BANKNIFTY08DEC22P42400
111111111111111
BANKNIFTY08DEC22P42400
14512145512451245845
orderupdate
{'t': 'om', 'norenordno': '22120700030420', 'uid': 'FA13620', 'actid': 'FA13620', 'exch': 'NFO', 'tsym': 'BANKNIFTY08DEC22C43900', 'trantype': 'S', 'qty': '25', 'prc': '0.00', 'pcode': 'I', 'remarks': 'mkt_order', 'status': 'PENDING', 'reporttype': 'NewAck', 'prctyp': 'MKT', 'ret': 'DAY', 'exchordid': '', 'dscqty': '0'}
orderupdate
{'t': 'om', 'norenordno': '22120700

KeyboardInterrupt: ignored

In [ ]:
# import datetime as dt 
# import pytz


# dt1 = dt.datetime.now()
# print(dt1 )


# dt2 = dt.datetime.now( pytz.timezone( "Asia/Calcutta" ) ).time()
# print(dt2)




# dt3 = dt.datetime.local_time()


# print(dt3)

In [ ]:
pprint.pprint(api.get_order_book())




order_book  = api.get_order_book()

if order_book[0]['status']  == 'REJECTED' :
  print( ' rejected order is this ')

In [24]:
print( ce_lp_data)
print( lp_data)

print(pe_lp_data)

{'43308': '11886.40', '43310': '11149.95', '43312': '10650.15', '43314': '10150.30', '43317': '9650.50', '43324': '9150.65', '43336': '8650.85', '43338': '8102.15', '43343': '7651.20', '43351': '7300.00', '43357': '6651.55', '43359': '6210.00', '43363': '5851.85', '43366': '5751.85', '43393': '5651.90', '43399': '5551.95', '43402': '5451.95', '43412': '5352.00', '43414': '5252.05', '43433': '5254.90', '43435': '5052.10', '43437': '4952.15', '43440': '4852.20', '43451': '4752.20', '43459': '4717.00', '43461': '4552.30', '43471': '4452.35', '43473': '4352.35', '43489': '4252.40', '43495': '4277.15', '43497': '4052.45', '43499': '3952.50', '43507': '3852.55', '43513': '3740.90', '43523': '3675.85', '43525': '3552.65', '43527': '3250.00', '43529': '2702.05', '43531': '3322.00', '43533': '3250.55', '43535': '3064.30', '43552': '2952.90', '43554': '2852.95', '43564': '2805.70', '43566': '2771.50', '43573': '2568.85', '43575': '2594.85', '43578': '2370.10', '43585': '2328.50', '43594': '2233.